In [1]:
import onnx
import torch
import json
import numpy as np
import os
import shutil
import warnings
from copy import deepcopy
from distutils.dir_util import copy_tree
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.transformation.general import SortGraph
from qonnx.transformation.merge_onnx_models import MergeONNXModels
from qonnx.transformation.insert_topk import InsertTopK
from qonnx.core.datatype import DataType
from qonnx.custom_op.registry import getCustomOp
from qonnx.transformation.bipolar_to_xnor import ConvertBipolarMatMulToXnorPopcount
from qonnx.transformation.fold_constants import FoldConstants
from qonnx.transformation.general import (
    ApplyConfig,
    GiveReadableTensorNames,
    GiveUniqueNodeNames,
    RemoveStaticGraphInputs,
    RemoveUnusedTensors,
)
from qonnx.transformation.infer_data_layouts import InferDataLayouts
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.lower_convs_to_matmul import LowerConvsToMatMul
from qonnx.transformation.make_input_chanlast import MakeInputChannelsLast
from qonnx.util.cleanup import cleanup as qonnx_cleanup
from qonnx.util.cleanup import cleanup_model
from qonnx.util.config import extract_model_config_to_json
from shutil import copy

import finn.transformation.fpgadataflow.convert_to_hw_layers as to_hw
import finn.transformation.streamline.absorb as absorb
from finn.analysis.fpgadataflow.dataflow_performance import dataflow_performance
from finn.analysis.fpgadataflow.exp_cycles_per_layer import exp_cycles_per_layer
from finn.analysis.fpgadataflow.hls_synth_res_estimation import hls_synth_res_estimation
from finn.analysis.fpgadataflow.op_and_param_counts import (
    aggregate_dict_keys,
    op_and_param_counts,
)
from finn.analysis.fpgadataflow.res_estimation import (
    res_estimation,
    res_estimation_complete,
)
from finn.builder.build_dataflow_config import (
    DataflowBuildConfig,
    DataflowOutputType,
    ShellFlowType,
    VerificationStepType,
)
from finn.core.onnx_exec import execute_onnx
from finn.core.rtlsim_exec import rtlsim_exec
from finn.core.throughput_test import throughput_test_rtlsim
from finn.transformation.fpgadataflow.annotate_cycles import AnnotateCycles
from finn.transformation.fpgadataflow.compile_cppsim import CompileCppSim
from finn.transformation.fpgadataflow.create_dataflow_partition import (
    CreateDataflowPartition,
)
from finn.transformation.fpgadataflow.create_stitched_ip import CreateStitchedIP
from finn.transformation.fpgadataflow.derive_characteristic import (
    DeriveCharacteristic,
    DeriveFIFOSizes,
)
from finn.transformation.fpgadataflow.hlssynth_ip import HLSSynthIP
from finn.transformation.fpgadataflow.insert_dwc import InsertDWC
from finn.transformation.fpgadataflow.insert_fifo import InsertFIFO
from finn.transformation.fpgadataflow.make_pynq_driver import MakePYNQDriver
from finn.transformation.fpgadataflow.make_zynq_proj import ZynqBuild
from finn.transformation.fpgadataflow.minimize_accumulator_width import (
    MinimizeAccumulatorWidth,
)
from finn.transformation.fpgadataflow.minimize_weight_bit_width import (
    MinimizeWeightBitWidth,
)
from finn.transformation.fpgadataflow.prepare_cppsim import PrepareCppSim
from finn.transformation.fpgadataflow.prepare_ip import PrepareIP
from finn.transformation.fpgadataflow.prepare_rtlsim import PrepareRTLSim
from finn.transformation.fpgadataflow.replace_verilog_relpaths import (
    ReplaceVerilogRelPaths,
)
from finn.transformation.fpgadataflow.set_exec_mode import SetExecMode
from finn.transformation.fpgadataflow.set_fifo_depths import (
    InsertAndSetFIFODepths,
    RemoveShallowFIFOs,
    SplitLargeFIFOs,
)
from finn.transformation.fpgadataflow.set_folding import SetFolding
from finn.transformation.fpgadataflow.specialize_layers import SpecializeLayers
from finn.transformation.fpgadataflow.synth_ooc import SynthOutOfContext
from finn.transformation.fpgadataflow.vitis_build import VitisBuild
from finn.transformation.move_reshape import RemoveCNVtoFCFlatten
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
from finn.transformation.qonnx.quant_act_to_multithreshold import (
    default_filter_function_generator,
)
from finn.transformation.streamline import Streamline
from finn.transformation.streamline.reorder import MakeMaxPoolNHWC
from finn.util.basic import (
    get_rtlsim_trace_depth,
    pyverilate_get_liveness_threshold_cycles,
)
from finn.util.pyverilator import verilator_fifosim
from finn.util.test import execute_parent

from finn.util.visualization import showInNetron
from finn.util.pytorch import ToTensor
from brevitas.onnx import export_qonnx

from finn.transformation.fpgadataflow import convert_to_hw_layers as convert
from qonnx.transformation.lower_convs_to_matmul import LowerConvsToMatMul
from finn.transformation.streamline.round_thresholds import RoundAndClipThresholds
%load_ext autoreload
%autoreload 2

In [7]:
model = ModelWrapper('streamlined_model.onnx')
showInNetron('streamlined_model.onnx')

Stopping http://0.0.0.0:8081
Serving 'streamlined_model.onnx' at http://0.0.0.0:8081


In [8]:
from qonnx.transformation.general import ConvertSubToAdd, ConvertDivToMul
import finn.transformation.streamline.collapse_repeated as collapse
import finn.transformation.streamline.reorder as reorder

model = model.transform(InferDataLayouts())
model = model.transform(convert.InferGlobalAccPoolLayer())
model = model.transform(convert.InferPool())
model = model.transform(absorb.AbsorbTransposeIntoFlatten())
model = model.transform(reorder.MoveScalarLinearPastInvariants())
model = model.transform(absorb.AbsorbMulIntoMultiThreshold())
model = model.transform(LowerConvsToMatMul())
model = model.transform(convert.InferChannelwiseLinearLayer())
model = model.transform(convert.InferConvInpGen())
model = model.transform(convert.InferQuantizedMatrixVectorActivation())
model = model.transform(convert.InferBinaryMatrixVectorActivation())

model = model.transform(absorb.AbsorbConsecutiveTransposes())
model = model.transform(reorder.MoveTransposePastFork())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(absorb.AbsorbConsecutiveTransposes())

model = model.transform(reorder.MoveTransposePastJoinAdd())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(reorder.MoveTransposePastFork())
model = model.transform(absorb.AbsorbConsecutiveTransposes())

model = model.transform(reorder.MoveTransposePastJoinAdd())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(reorder.MoveTransposePastFork())
model = model.transform(absorb.AbsorbConsecutiveTransposes())

In [10]:
model = model.transform(reorder.MoveTransposePastJoinAdd())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(reorder.MoveTransposePastFork())
model = model.transform(absorb.AbsorbConsecutiveTransposes())

model = model.transform(reorder.MoveTransposePastJoinAdd())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(reorder.MoveTransposePastFork())
model = model.transform(absorb.AbsorbConsecutiveTransposes())

In [12]:
model = model.transform(reorder.MoveTransposePastJoinAdd())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(reorder.MoveTransposePastFork())
model = model.transform(absorb.AbsorbConsecutiveTransposes())

In [14]:
model = model.transform(reorder.MoveTransposePastJoinAdd())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(reorder.MoveTransposePastFork())
model = model.transform(absorb.AbsorbConsecutiveTransposes())

In [15]:
model.save('model_2.onnx')
showInNetron('model_2.onnx')

Stopping http://0.0.0.0:8081
Serving 'model_2.onnx' at http://0.0.0.0:8081


In [17]:
#model = model.transform(InferDataLayouts())
#model = model.transform(RoundAndClipThresholds())
#model = model.transform(convert.InferThresholdingLayer())

#model = model.transform(RemoveCNVtoFCFlatten())
#model = model.transform(InferDataLayouts())
#model = model.transform(convert.InferLabelSelectLayer())

#model = tidy_up(model)
model = model.transform(convert.InferAddStreamsLayer())
model = model.transform(convert.InferDuplicateStreamsLayer())

/srv/homes/ipanagou/thesis/finn/src/finn/custom_op/fpgadataflow/thresholding.py:84: UserWarning: inputDataType changing for Thresholding_: UINT32 -> UINT5 
  warnings.warn(warn_str)


In [ ]:
model.save('model_3.onnx')
showInNetron('model_3.onnx')

In [ ]:
model = model.transform(reorder.MoveTransposePastJoinAdd())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(reorder.MoveTransposePastFork())
model = model.transform(absorb.AbsorbConsecutiveTransposes())

model = model.transform(reorder.MoveTransposePastJoinAdd())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(reorder.MoveTransposePastFork())
model = model.transform(absorb.AbsorbConsecutiveTransposes())

Serving 'model_2.onnx' at http://0.0.0.0:8081


In [2]:
def preprocessing(model):
	input_shape = model.get_tensor_shape(model.graph.input[0].name)
	preproc = ToTensor()
	export_qonnx(preproc, torch.randn(input_shape), "preproc.onnx", opset_version = 11)
	qonnx_cleanup("preproc.onnx", out_file = "preproc.onnx")
	preproc_model = ModelWrapper("preproc.onnx")
	preproc_model = preproc_model.transform(ConvertQONNXtoFINN())

	model = model.transform(MergeONNXModels(preproc_model))
	global_inp_name = model.graph.input[0].name
	model.set_tensor_datatype(global_inp_name, DataType["UINT8"])
	model = model.transform(InferShapes())
	model = model.transform(FoldConstants())
	model = model.transform(GiveUniqueNodeNames())
	model = model.transform(GiveReadableTensorNames())
	model = model.transform(InferDataTypes())
	model = model.transform(RemoveStaticGraphInputs())
	return model

def postprocessing(model):
	model = model.transform(InsertTopK(k=1))
	model = model.transform(InferShapes())
	model = model.transform(FoldConstants())
	model = model.transform(GiveUniqueNodeNames())
	model = model.transform(GiveReadableTensorNames())
	model = model.transform(InferDataTypes())
	model = model.transform(RemoveStaticGraphInputs())
	return model

def qonnx_to_finn(model):
    """
    This step will only execute if QONNX nodes are found.
    These include the following op_types: "Quant" , "Trunc" and "BinaryQuant".
    If such nodes are found the step will run the tidy-up step from QONNX
    and then convert the QONNX model to the FINN-ONNX dialect.
    """
    # Check if any QONNX nodes exist, i.e. BinaryQuant, Quant or Trunc
    q_count = 0
    for op_type in ["BinaryQuant", "Quant", "Trunc"]:
        q_count += len(model.get_nodes_by_op_type(op_type))
    if q_count == 0:
        return model

    # QONNX cleanup
    model = cleanup_model(model)
    # QONNX to FINN-ONNX
    model = model.transform(
        ConvertQONNXtoFINN(
            filter_function=default_filter_function_generator(
                max_multithreshold_bit_width=8
            )
        )
    )
    return model


def tidy_up(model):
    """Run the tidy-up step on given model. This includes shape and datatype
    inference, constant folding, and giving nodes and tensors better names.
    """

    model = model.transform(InferShapes())
    model = model.transform(FoldConstants())
    model = model.transform(GiveUniqueNodeNames())
    model = model.transform(GiveReadableTensorNames())
    model = model.transform(InferDataTypes())
    model = model.transform(RemoveStaticGraphInputs())

    return model

In [34]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision.models.resnet import BasicBlock, Bottleneck
from torch import Tensor

# Load the original ResNet model
original_resnet = models.resnet18()
#original_resnet.train()

def add_relu_after_bn():
    BasicBlock.forward = basic_block_forward
    '''
    module_output = module
    if isinstance(module, nn.BatchNorm2d) and (name == "bn2" or name == "bn3"):
        module_output = nn.Sequential(module, nn.ReLU())
    for name, child in module.named_children():
        module_output.add_module(name, add_relu_after_bn(name, child))
    del module

    return module_output
    '''


def basic_block_forward(self, x: Tensor) -> Tensor:
    identity = x
    out = self.conv1(x)
    out = self.bn1(out)
    out = self.relu(out)

    out = self.conv2(out)
    out = self.bn2(out)
    out = self.relu(out)

    if self.downsample is not None:
        identity = self.relu(self.downsample(x))

    out = out + identity
    out = self.relu(out)

    return out
    
# Create a custom ResNet with ReLU inserted before addition operations
add_relu_after_bn()

print(original_resnet)
name = f'model_with_relus.onnx'
torch.onnx.export(original_resnet, ref_input, name, export_params=True, verbose = True, opset_version=11)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [35]:
showInNetron('model_with_relus.onnx')

Stopping http://0.0.0.0:8081
Serving 'model_with_relus.onnx' at http://0.0.0.0:8081


In [36]:
showInNetron('model_1.0_0.0.onnx')

Stopping http://0.0.0.0:8081
Serving 'model_1.0_0.0.onnx' at http://0.0.0.0:8081


In [37]:
model = ModelWrapper('model_1.0_0.0_quant.onnx')
showInNetron('model_1.0_0.0_quant.onnx')

Stopping http://0.0.0.0:8081
Serving 'model_1.0_0.0_quant.onnx' at http://0.0.0.0:8081


In [38]:
model = preprocessing(model)
model = postprocessing(model)
model = model.transform(MakeInputChannelsLast())
model = tidy_up(model)
model = qonnx_to_finn(model)
model = tidy_up(model)
model.save('model.onnx')
showInNetron('model.onnx')

/srv/homes/ipanagou/thesis/finn/deps/qonnx/src/qonnx/transformation/infer_data_layouts.py:127: UserWarning: Assuming 4D input is NCHW
  warnings.warn("Assuming 4D input is NCHW")
/srv/homes/ipanagou/thesis/finn/deps/qonnx/src/qonnx/transformation/gemm_to_matmul.py:57: UserWarning: The GemmToMatMul transformation only offers explicit support for version 9 of the Gemm node, but the ONNX version of the supplied model is 11. Thus the transformation may fail or return incomplete results.
  warnings.warn(


Stopping http://0.0.0.0:8081
Serving 'model.onnx' at http://0.0.0.0:8081


In [39]:
# Streamlining

from qonnx.transformation.general import ConvertSubToAdd, ConvertDivToMul
import finn.transformation.streamline.collapse_repeated as collapse
import finn.transformation.streamline.reorder as reorder

model = model.transform(ConvertSubToAdd())
model = model.transform(ConvertDivToMul())
model = model.transform(collapse.CollapseRepeatedMul())
model = model.transform(absorb.AbsorbSignBiasIntoMultiThreshold())
model = model.transform(absorb.AbsorbAddIntoMultiThreshold())
model = model.transform(absorb.AbsorbMulIntoMultiThreshold())
model = model.transform(absorb.AbsorbScalarMulAddIntoTopK())

model = model.transform(reorder.MoveMulPastMaxPool())
model = model.transform(reorder.MoveLinearPastFork())
model = model.transform(reorder.MoveLinearPastEltwiseAdd())
model = model.transform(reorder.MoveScalarMulPastConv())
model = model.transform(reorder.MoveScalarMulPastMatMul())
model = model.transform(reorder.MoveScalarLinearPastInvariants()) # for the mul before the global average pool
model = model.transform(absorb.AbsorbMulIntoMultiThreshold())
model = model.transform(absorb.AbsorbScalarMulAddIntoTopK())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())

model = model.transform(RoundAndClipThresholds())

model.save('model.onnx')
showInNetron('model.onnx')

found topk node
found Mul node
0.26739833
it is actually scalar
found topk node
found topk node
Stopping http://0.0.0.0:8081
Serving 'model.onnx' at http://0.0.0.0:8081


In [40]:
# CONVERT TO HW
model = model.transform(InferDataLayouts())
model = model.transform(convert.InferGlobalAccPoolLayer())
model = model.transform(convert.InferPool())
model = model.transform(absorb.AbsorbTransposeIntoFlatten())
model = model.transform(reorder.MoveScalarLinearPastInvariants())
model = model.transform(absorb.AbsorbMulIntoMultiThreshold())
model = model.transform(LowerConvsToMatMul())
model = model.transform(convert.InferChannelwiseLinearLayer())
model = model.transform(convert.InferConvInpGen())
model = model.transform(convert.InferQuantizedMatrixVectorActivation())

model = model.transform(absorb.AbsorbConsecutiveTransposes())
model = model.transform(reorder.MoveTransposePastFork())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(absorb.AbsorbConsecutiveTransposes())

model = model.transform(reorder.MoveTransposePastJoinAdd())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(reorder.MoveTransposePastFork())
model = model.transform(absorb.AbsorbConsecutiveTransposes())

model = model.transform(reorder.MoveTransposePastJoinAdd())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(reorder.MoveTransposePastFork())
model = model.transform(absorb.AbsorbConsecutiveTransposes())

model = model.transform(InferDataLayouts())
model = model.transform(convert.InferThresholdingLayer())

model = model.transform(RemoveCNVtoFCFlatten())
model = model.transform(InferDataLayouts())
model = model.transform(convert.InferLabelSelectLayer())

model = tidy_up(model)
model = model.transform(convert.InferAddStreamsLayer())
model = model.transform(convert.InferDuplicateStreamsLayer())

model.save('model.onnx')
showInNetron('model.onnx')

INT8
-128


/srv/homes/ipanagou/thesis/finn/src/finn/custom_op/fpgadataflow/thresholding.py:84: UserWarning: inputDataType changing for Thresholding_4: UINT32 -> UINT9 
  warnings.warn(warn_str)
/srv/homes/ipanagou/thesis/finn/src/finn/custom_op/fpgadataflow/thresholding.py:84: UserWarning: inputDataType changing for Thresholding_7: UINT32 -> UINT9 
  warnings.warn(warn_str)
/srv/homes/ipanagou/thesis/finn/src/finn/custom_op/fpgadataflow/thresholding.py:84: UserWarning: inputDataType changing for Thresholding_11: UINT32 -> UINT9 
  warnings.warn(warn_str)
/srv/homes/ipanagou/thesis/finn/src/finn/custom_op/fpgadataflow/thresholding.py:84: UserWarning: inputDataType changing for Thresholding_14: UINT32 -> UINT9 
  warnings.warn(warn_str)
/srv/homes/ipanagou/thesis/finn/src/finn/custom_op/fpgadataflow/thresholding.py:84: UserWarning: inputDataType changing for Thresholding_18: UINT32 -> UINT9 
  warnings.warn(warn_str)
/srv/homes/ipanagou/thesis/finn/src/finn/custom_op/fpgadataflow/thresholding.py:84

Stopping http://0.0.0.0:8081
Serving 'model.onnx' at http://0.0.0.0:8081


In [98]:
import finn.transformation.streamline.reorder as reorder



#model.save('model.onnx')
#showInNetron('model.onnx')

found topk node


In [103]:
model.save('model.onnx')
showInNetron('model.onnx')

Stopping http://0.0.0.0:8081
Serving 'model.onnx' at http://0.0.0.0:8081


In [99]:
from copy import deepcopy

streamlined = deepcopy(model)

In [138]:
model = model.transform(InferDataLayouts()) # infer data layouts is a lifesaver
model = model.transform(convert.InferGlobalAccPoolLayer())

model.save('model.onnx')
showInNetron('model.onnx')

Stopping http://0.0.0.0:8081
Serving 'model.onnx' at http://0.0.0.0:8081


In [139]:
model = model.transform(convert.InferPool())

model.save('model.onnx')
showInNetron('model.onnx')

Stopping http://0.0.0.0:8081
Serving 'model.onnx' at http://0.0.0.0:8081


In [140]:
model = model.transform(absorb.AbsorbTransposeIntoFlatten())
model.save('model.onnx')
showInNetron('model.onnx')

Stopping http://0.0.0.0:8081
Serving 'model.onnx' at http://0.0.0.0:8081


In [141]:
model = model.transform(reorder.MoveScalarLinearPastInvariants())
model.save('model.onnx')
showInNetron('model.onnx')

Stopping http://0.0.0.0:8081
Serving 'model.onnx' at http://0.0.0.0:8081


In [142]:
model = model.transform(absorb.AbsorbMulIntoMultiThreshold())
model.save('model.onnx')
showInNetron('model.onnx')

Stopping http://0.0.0.0:8081
Serving 'model.onnx' at http://0.0.0.0:8081


In [143]:
model = model.transform(LowerConvsToMatMul())
model = model.transform(convert.InferChannelwiseLinearLayer())
model = model.transform(convert.InferConvInpGen())
model = model.transform(convert.InferQuantizedMatrixVectorActivation())
model.save('model.onnx')
showInNetron('model.onnx')

INT8
-128
Stopping http://0.0.0.0:8081
Serving 'model.onnx' at http://0.0.0.0:8081


In [145]:

#model.save('model.onnx')
#showInNetron('model.onnx')

In [146]:


#model.save('model.onnx')
#showInNetron('model.onnx')

In [147]:


#model.save('model.onnx')
#showInNetron('model.onnx')

In [148]:

model.save('model.onnx')
showInNetron('model.onnx')

Stopping http://0.0.0.0:8081
Serving 'model.onnx' at http://0.0.0.0:8081


In [149]:


model.save('model.onnx')
showInNetron('model.onnx')

Stopping http://0.0.0.0:8081
Serving 'model.onnx' at http://0.0.0.0:8081


In [150]:


model.save('model.onnx')
showInNetron('model.onnx')

Stopping http://0.0.0.0:8081
Serving 'model.onnx' at http://0.0.0.0:8081


In [151]:

model.save('model.onnx')
showInNetron('model.onnx')

/srv/homes/ipanagou/thesis/finn/src/finn/custom_op/fpgadataflow/thresholding.py:84: UserWarning: inputDataType changing for Thresholding_MultiThreshold_11: INT32 -> INT9 
  warnings.warn(warn_str)
/srv/homes/ipanagou/thesis/finn/src/finn/custom_op/fpgadataflow/thresholding.py:84: UserWarning: inputDataType changing for Thresholding_MultiThreshold_18: INT32 -> INT9 
  warnings.warn(warn_str)
/srv/homes/ipanagou/thesis/finn/src/finn/custom_op/fpgadataflow/thresholding.py:84: UserWarning: inputDataType changing for Thresholding_MultiThreshold_25: INT32 -> INT9 
  warnings.warn(warn_str)


Stopping http://0.0.0.0:8081
Serving 'model.onnx' at http://0.0.0.0:8081


In [152]:

model.save('model.onnx')
showInNetron('model.onnx')

Stopping http://0.0.0.0:8081
Serving 'model.onnx' at http://0.0.0.0:8081


In [157]:
model = tidy_up(model)
model = model.transform(convert.InferAddStreamsLayer())


input: "Thresholding_3_out0"
input: "DuplicateStreams_0_out1"
output: "Add_0_out0"
name: "Add_0"
op_type: "Add"

INT8
UINT8
different data types
input: "Thresholding_6_out0"
input: "DuplicateStreams_1_out1"
output: "Add_1_out0"
name: "Add_1"
op_type: "Add"

INT8
UINT8
different data types
input: "Thresholding_13_out0"
input: "DuplicateStreams_3_out1"
output: "Add_2_out0"
name: "Add_2"
op_type: "Add"

INT8
UINT8
different data types
input: "Thresholding_20_out0"
input: "DuplicateStreams_5_out1"
output: "Add_3_out0"
name: "Add_3"
op_type: "Add"

INT8
UINT8
different data types
input: "Thresholding_27_out0"
input: "DuplicateStreams_7_out1"
output: "Add_4_out0"
name: "Add_4"
op_type: "Add"

INT8
UINT8
different data types
Stopping http://0.0.0.0:8081
Serving 'model.onnx' at http://0.0.0.0:8081


In [ ]:
model = model.transform(reorder.MoveTransposePastJoinAdd())
#model.save('model.onnx')
#showInNetron('model.onnx')

In [ ]:
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
#model.save('model.onnx')
#showInNetron('model.onnx')

In [ ]:
model = model.transform(reorder.MoveTransposePastFork())
#model.save('model.onnx')
#showInNetron('model.onnx')

In [ ]:
model = model.transform(absorb.AbsorbConsecutiveTransposes())
model = model.transform(reorder.MoveTransposePastJoinAdd())
#model.save('model.onnx')
#showInNetron('model.onnx')

In [ ]:
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
#model.save('model.onnx')
#showInNetron('model.onnx')

In [ ]:
model = model.transform(reorder.MoveTransposePastFork())
model = model.transform(absorb.AbsorbConsecutiveTransposes())
#model.save('model.onnx')
#showInNetron('model.onnx')

In [ ]:
model = model.transform(convert.InferPool())
#model.save('model.onnx')
#showInNetron('model.onnx')

In [ ]:
model = model.transform(absorb.AbsorbConsecutiveTransposes())
#model.save('model.onnx')
#showInNetron('model.onnx')

In [ ]:
model = model.transform(reorder.MoveTransposePastFork())
model = model.transform(absorb.AbsorbConsecutiveTransposes())
#model.save('model.onnx')
#showInNetron('model.onnx')

In [ ]:
model = model.transform(LowerConvsToMatMul())
model = model.transform(convert.InferChannelwiseLinearLayer())
model = model.transform(convert.InferConvInpGen())
model = model.transform(convert.InferQuantizedMatrixVectorActivation())
model = model.transform(absorb.AbsorbConsecutiveTransposes())
model.save('model.onnx')
showInNetron('model.onnx')

In [ ]:
orig_model = deepcopy(model)

In [ ]:
model = model.transform(reorder.MoveTransposePastJoinAdd())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(reorder.MoveTransposePastFork())
model = model.transform(absorb.AbsorbConsecutiveTransposes())
model.save('model.onnx')
showInNetron('model.onnx')

In [ ]:
model = model.transform(reorder.MoveTransposePastJoinAdd())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(reorder.MoveTransposePastFork())
model = model.transform(absorb.AbsorbConsecutiveTransposes())
model.save('model.onnx')
showInNetron('model.onnx')

In [ ]:
model = model.transform(convert.InferAddStreamsLayer())
model = model.transform(convert.InferDuplicateStreamsLayer())
model = model.transform(convert.InferThresholdingLayer())
model = model.transform(RemoveCNVtoFCFlatten())
model = model.transform(convert.InferLabelSelectLayer())
model.save('model.onnx')
showInNetron('model.onnx')